In [3]:
import os

# MODEL = "mixtral:8x7b"
MODEL = "llama2"


In [4]:
from langchain_community.llms import Ollama
# from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_openai.embeddings import OpenAIEmbeddings


model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")


"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that brought a smile to your face! Do you want to hear another one?"

In [7]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [8]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer to the question "What\'s your name?" would be: "My name is Santiago."'

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data_science_interview_prep.pdf")
pages = loader.load_and_split()
pages


[Document(page_content='1', metadata={'source': 'day_trading_for_dummies.pdf', 'page': 1}),
 Document(page_content='2Day Trading For Dummies®, 3rd Edition\nPublished by: John Wiley & Sons, Inc., 111 River Street, Hoboken, NJ 07030-5774,\nwww.wiley.com\nCopyright © 2014 by John Wiley & Sons, Inc., Hoboken, New Jersey\nPublished simultaneously in Canada\nNo part of this publication may be reproduced, stored in a retrieval system or transmitted in any form or\nby any means, electronic, mechanical, photocopying, recording, scanning or otherwise, except as\npermitted under Sections 107 or 108 of the 1976 United States Copyright Act, without the prior written\npermission of the Publisher. Requests to the Publisher for permission should be addressed to the\nPermissions Department, John Wiley & Sons, Inc., 111 River Street, Hoboken, NJ 07030, (201) 748-\n6011, fax (201) 748-6008, or online at http://www.wiley.com/go/permissions.\nTrademarks: Wiley, For Dummies, the Dummies Man logo, Dummies.co

In [12]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

In [ ]:
chain.batch([{"question": q} for q in questions])

In [ ]:
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s, end="", flush=True)
